In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_id = test['id']

In [35]:
test = test.drop(['id'] , axis=1)


In [36]:
test

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,1,1,1,9500,1,1,141.0,1,3,1,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,1,1,1,9238,1,1,128.0,1,1,19,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,1,1,1,9238,1,1,118.0,1,1,19,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,1,44,1,9147,1,39,130.0,1,1,19,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,1,39,1,9670,1,1,110.0,1,1,37,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,1,1,2,171,1,1,128.0,1,38,37,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,2,39,1,9119,1,19,133.1,1,19,37,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,1,1,1,171,1,1,127.0,1,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,1,1,3,9773,1,1,132.0,1,19,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [52]:
data.columns


Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd s

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def data_preprocessing (data):
    data = data.drop(columns = ["id"])
    data["Target"] = le.fit_transform(data["Target"])
    return data

data = data_preprocessing(data)


In [39]:
from sklearn.model_selection import train_test_split
X = data.drop(columns = ["Target"])
y = data["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler , RobustScaler
scaler =RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

#model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
from xgboost import XGBClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize the XGBoost classifier
xgb_model = XGBClassifier(random_state=42)

# Initialize Grid Search
grid_search = RandomizedSearchCV(estimator=xgb_model,param_distributions=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters
model = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    random_state=42
)

model.fit(X_train , y_train)
sfm = SelectFromModel(model, threshold='mean')

# Fit the transformer
sfm.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=30, n_estimators=50, subsample=0.9; total time=  30.3s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=30, n_estimators=50, subsample=0.9; total time=  30.8s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=30, n_estimators=50, subsample=0.9; total time=  30.8s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=30, n_estimators=50, subsample=0.9; total time=  31.0s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=30, n_estimators=50, subsample=0.9; total time=  30.8s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=20, n_estimators=50, subsample=0.9; total time=  20.4s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=20, n_estimators=50, subsample=0.9; total time=  20.4s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=20, n_estimators=50, subsample=0.9; total time=  22.4s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=20, n_estimators=10

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
model.score(X_test , y_test)

0.8326581285938317

In [46]:
test = scaler.transform(test)

In [47]:
y_predicted = model.predict(test)
y_predicted

array([0, 2, 2, ..., 0, 0, 0])

In [48]:
y_predicted = le.inverse_transform(y_predicted)

In [49]:
y_predicted[000:700]

array(['Dropout', 'Graduate', 'Graduate', 'Dropout', 'Enrolled',
       'Graduate', 'Graduate', 'Graduate', 'Dropout', 'Graduate',
       'Graduate', 'Graduate', 'Graduate', 'Enrolled', 'Dropout',
       'Enrolled', 'Graduate', 'Graduate', 'Enrolled', 'Graduate',
       'Graduate', 'Dropout', 'Graduate', 'Enrolled', 'Dropout',
       'Dropout', 'Dropout', 'Graduate', 'Graduate', 'Enrolled',
       'Graduate', 'Graduate', 'Dropout', 'Enrolled', 'Dropout',
       'Dropout', 'Dropout', 'Graduate', 'Graduate', 'Graduate',
       'Graduate', 'Graduate', 'Graduate', 'Graduate', 'Enrolled',
       'Graduate', 'Dropout', 'Dropout', 'Enrolled', 'Graduate',
       'Graduate', 'Graduate', 'Dropout', 'Dropout', 'Graduate',
       'Enrolled', 'Enrolled', 'Enrolled', 'Graduate', 'Graduate',
       'Enrolled', 'Graduate', 'Dropout', 'Graduate', 'Dropout',
       'Graduate', 'Enrolled', 'Dropout', 'Graduate', 'Enrolled',
       'Enrolled', 'Graduate', 'Graduate', 'Graduate', 'Dropout',
       'Enrolle

In [50]:
submission = pd.DataFrame({"id":test_id , "Target": y_predicted})

In [51]:
submission.to_csv("submission7.csv" , index=False)